# Helix LLM & Embedding Providers

In [2]:
from helix.client import Client
from helix.mcp import MCPServer, ToolConfig
from pydantic import BaseModel

import nest_asyncio
nest_asyncio.apply() 

class Person(BaseModel):
    name: str
    age: int
    occupation: str

In [ ]:
# Helix Client
helix_client = Client(local=True)

# Helix MCP Server
# Enable search_vector tool instead because we will add the embedder later
mcp_server = MCPServer("helix-mcp", helix_client, tool_config=ToolConfig(search_vector=True, search_vector_text=False))

# Run mcp server in background thread on port 8000
# To stop the server, restart the notebook
mcp_server.run_bg()

## OpenAI

In [ ]:
from helix.providers.openai_client import OpenAIProvider
from helix.embedding.openai_client import OpenAIEmbedder

openai_llm = OpenAIProvider(
    name="openai-llm",
    instructions="You are a helpful assistant.",
    model="gpt-5-nano",
    reasoning={
        "effort":"low"
    },
    history=True
)

openai_embedder = OpenAIEmbedder()

# Set embedder for MCP server
mcp_server.embedder = openai_embedder

[08/25/25 15:37:12] INFO     Starting MCP server 'helix-mcp' with transport 'streamable-http' on     ]8;id=346993;file:///Users/gwuwong/Documents/helixdb/helix-py/.venv/lib/python3.13/site-packages/fastmcp/server/server.py\server.py]8;;\:]8;id=229961;file:///Users/gwuwong/Documents/helixdb/helix-py/.venv/lib/python3.13/site-packages/fastmcp/server/server.py#1519\1519]8;;\
                             http://127.0.0.1:8000/mcp/                                                            

### Generating LLM Responses

In [5]:
# Generating response from string
response = openai_llm.generate("Hello, how are you?")
print(response, '\n' + '-'*100, '\n')

# Generate response from message
response = openai_llm.generate([{"role": "user", "content": "My name is John Berger. I am a software engineer. I am 25 years old."}])
print(response, '\n' + '-'*100, '\n')

# Generate structured response
response = openai_llm.generate([{"role": "user", "content": "Who am I?"}], Person)
print(response, '\n' + '-'*100, '\n')

# Enable MCP tools
openai_llm.enable_mcps("helix-mcp")
response = openai_llm.generate([{"role": "user", "content": "What MCP tools do you have?"}])
print(response, '\n' + '-'*100, '\n')

Hi! I’m here and ready to help. How can I assist you today? 
---------------------------------------------------------------------------------------------------- 

Nice to meet you, John. Thanks for sharing—25 and a software engineer. What would you like to dive into today? I can help with coding questions, debugging, project ideas, optimization, interview prep, resume tips, or anything specific you’re working on. If you have a problem or a project, tell me the details and I’ll jump in. 
---------------------------------------------------------------------------------------------------- 

name='John Berger' age=25 occupation='Software Engineer' 
---------------------------------------------------------------------------------------------------- 

I have the MCP traversal tools available. Here’s a quick rundown of what they do and how you can use them:

What you can do with the MCP tools
- init(): Start a traversal session. Returns a connection_id you’ll use for subsequent calls.
- next

### Generating Embeddings

In [6]:
# Embedding text
vector = openai_embedder.embed("My name is John Berger. I am a software engineer. I am 25 years old.")
print(f"Dimension: {len(vector)}\t" + str(vector[:5]), "\n")

# Embedding batch documents
documents = ["My name is John Berger.", "I am a software engineer.", "I am 25 years old."]
for vector in openai_embedder.embed_batch(documents):
    print(f"Dimension: {len(vector)}\t" + str(vector[:5]))

Dimension: 1536	[0.0037015017587691545, 0.008918856270611286, -0.04589862376451492, 0.0055170003324747086, -0.025452231988310814] 



[HELIX] Embedding: 100%|██████████| 3/3 [00:00<00:00, 48960.75it/s]

Dimension: 1536	[0.009038523770868778, 0.024945339187979698, -0.05953341722488403, 0.013305002823472023, -0.06579749286174774]
Dimension: 1536	[0.0016707692993804812, -0.012029538862407207, -0.04344000294804573, -0.017710154876112938, 0.013520379550755024]
Dimension: 1536	[0.021052706986665726, 0.022584712132811546, -0.02384490892291069, 0.00512727303430438, -0.022152289748191833]


## Gemini

In [ ]:
from helix.providers.gemini_client import GeminiProvider
from helix.embedding.gemini_client import GeminiEmbedder

gemini_llm = GeminiProvider(
    model="gemini-2.0-flash",
    temperature=0.1,
    history=True
)

gemini_embedder = GeminiEmbedder()

# Set embedder for MCP server
mcp_server.embedder = gemini_embedder
mcp_server.embedder_args = {"task_type": "RETRIEVAL_QUERY"}

### Generating LLM Responses

In [8]:
# Generating response from string
response = gemini_llm.generate("Hello, how are you?")
print(response, '\n' + '-'*100, '\n')

# Generate response from message
response = gemini_llm.generate([{"role": "user", "parts": [{"text": "My name is John Berger. I am a software engineer. I am 25 years old."}]}])
print(response, '\n' + '-'*100, '\n')

# Generate structured response
response = gemini_llm.generate([{"role": "user", "parts": [{"text": "Who am I?"}]}], Person)
print(response, '\n' + '-'*100, '\n')

# Enable MCP tools
gemini_llm.enable_mcps("helix-mcp")
response = gemini_llm.generate([{"role": "user", "parts": [{"text": "What MCP tools do you have?"}]}])
print(response, '\n' + '-'*100, '\n')

I am doing well, thank you for asking! As a large language model, I don't experience emotions or feelings like humans do, but I am functioning optimally and ready to assist you. How can I help you today?
 
---------------------------------------------------------------------------------------------------- 

Okay, John Berger, 25-year-old software engineer. Nice to meet you! Is there anything specific you'd like to talk about or any way I can help you today? Perhaps you have a coding question, want to brainstorm ideas, or just want to chat? Let me know!
 
---------------------------------------------------------------------------------------------------- 

name='John Berger' age=25 occupation='software engineer' 
---------------------------------------------------------------------------------------------------- 

I have access to the following tools for interacting with the MCP:

*   `init`: Initializes the MCP traversal connection.
*   `next`: Gets the next item in the traversal resul

### Generating Embeddings

In [9]:
# Embedding text
vector = gemini_embedder.embed("My name is John Berger. I am a software engineer. I am 25 years old.", task_type="RETRIEVAL_DOCUMENT")
print(f"Dimension: {len(vector)}\t" + str(vector[:5]), "\n")

# Embedding batch documents
documents = ["My name is John Berger.", "I am a software engineer.", "I am 25 years old."]
for vector in gemini_embedder.embed_batch(documents, task_type="RETRIEVAL_DOCUMENT"):
    print(f"Dimension: {len(vector)}\t" + str(vector[:5]))

Dimension: 1536	[-0.017157292, 0.018253414, 0.017877506, -0.0758527, -0.013305571] 



[HELIX] Embedding: 100%|██████████| 3/3 [00:00<00:00, 43539.49it/s]

Dimension: 1536	[-0.010100627, 0.00897918, 0.015125714, -0.085755296, -0.018233819]
Dimension: 1536	[0.002266545, 0.0043500746, 0.0003194056, -0.100754924, -0.0135422405]
Dimension: 1536	[-0.013887029, 0.008958238, 0.0064157364, -0.08546713, -0.004919313]


## Anthropic

In [ ]:
from helix.providers.anthropic_client import AnthropicProvider
from helix.embedding.voyageai_client import VoyageAIEmbedder

anthropic_llm = AnthropicProvider(
    model="claude-3-5-haiku-20241022",
    temperature=0.1,
    history=True
)

voyageai_embedder = VoyageAIEmbedder()

# Set embedder for MCP server
mcp_server.embedder = voyageai_embedder
mcp_server.embedder_args = {"input_type": "query"}

### Generating LLM Responses

In [11]:
# Generating response from string
response = anthropic_llm.generate("Hello, how are you?")
print(response, '\n' + '-'*100, '\n')

# Generate response from message
response = anthropic_llm.generate([{"role": "user", "content": "My name is John Berger. I am a software engineer. I am 25 years old."}])
print(response, '\n' + '-'*100, '\n')

# Generate structured response
response = anthropic_llm.generate([{"role": "user", "content": "Who am I?"}], Person)
print(response, '\n' + '-'*100, '\n')

# Anthropic doesn't currently support local MCP servers via streamable http connection.

I'm doing well, thank you for asking! How are you today? Is there anything I can help you with? 
---------------------------------------------------------------------------------------------------- 

Hello John! It's nice to meet you. As a 25-year-old software engineer, I imagine you work on some interesting technical projects. Is there anything specific you'd like to discuss today? 
---------------------------------------------------------------------------------------------------- 

name='John Berger' age=25 occupation='software engineer' 
---------------------------------------------------------------------------------------------------- 



### Generating Embeddings

In [13]:
# Embedding text
vector = voyageai_embedder.embed("My name is John Berger. I am a software engineer. I am 25 years old.", input_type="document")
print(f"Dimension: {len(vector)}\t" + str(vector[:5]), "\n")

# Embedding batch documents
documents = ["My name is John Berger.", "I am a software engineer.", "I am 25 years old."]
for vector in voyageai_embedder.embed_batch(documents, input_type="document"):
    print(f"Dimension: {len(vector)}\t" + str(vector[:5]))

Dimension: 1024	[-0.017457768321037292, 0.04947829991579056, -0.029827343299984932, 0.0845692902803421, 0.04526738077402115] 



[HELIX] Embedding: 100%|██████████| 3/3 [00:00<00:00, 41120.63it/s]

Dimension: 1024	[-0.03412482887506485, 0.03371741250157356, -0.04263145104050636, 0.06879756599664688, 0.06010128930211067]
Dimension: 1024	[-0.00386068201623857, 0.05951542779803276, 0.010549462400376797, 0.05377177521586418, 0.02488693594932556]
Dimension: 1024	[-0.007679720874875784, 0.035258837044239044, -0.024098483845591545, 0.06100286915898323, 0.03362797945737839]
